In [ ]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP

Mounted at /gdrive
/gdrive/My Drive/NLP


In [ ]:
%pip install transformers datasets

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 298 kB 48.5 MB/s 
     |████████████████████████████████| 61 kB 469 kB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
     |████████████████████████████████| 3.3 MB 21.1 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 132 kB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 243 kB 43.8 MB/s 
     |████████████████████████████████| 160 kB 50.5 MB/s 
     |████████████████████████████████| 192 kB 49.2 MB/s 
     |████████████████████████████████| 271 kB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

%pip install bert_score

     |████████████████████████████████| 59 kB 3.2 MB/s 


In [ ]:
!nvidia-smi

Wed Dec  8 04:05:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python run_mlm.py --output_dir=models/cnn --overwrite_output_dir --model_type=bert --model_name_or_path=bert-base-cased \
  --do_train --train_file=data/cnn/train.txt --validation_file=data/cnn/valid.txt --do_eval --per_device_train_batch_size=2 \
  --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

12/08/2021 04:08:40 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/08/2021 04:08:40 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at

# Elon Musk


In [ ]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --repetition_penalty 2.0\
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 21:57:41 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=2.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
u you they, want if only Only - - - - + plus minus o : & and And For In All all of Of Now Todayday Tonight Day day today eve afternoon evening morning night on O
=== GENERATED SEQUENCE 2 ===
ays Alwaysless there there There 'ES s em Em out it a an ah uh er e EEOAAAICCSUSUSUUILADASADSDWDXL L S F D TT
=== GENERATED SEQUENCE 3 ===
nd & And … Ice ice by By Had the Thetheftfted to To Tendslaslasts s,, ( 1 1 )e mMpmamamamiamnam
=== GENERATED SEQUENCE 4 ===
 O S R T M F L V G CC W P N & e t g i o m r f u U u en En At at art Art Arts Science Sciences So to would but 

In [ ]:
!python run_metrics.py \
  --model_type bert \
  --model_name_or_path "models/elonmusk" \
  --test_data_file "data/elonmusk/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 22:00:01 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='bert', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/elonmusk/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head





# Karpathy

In [ ]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/karpathy" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --repetition_penalty 2.0\
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 21:39:05 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=2.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
[ ] } } } ) ) ) ) (': p a - - o O & and And + minus = to ssstatcalcalmalmsmtdvw w W 
=== GENERATED SEQUENCE 2 ===
!!!!!!!!!!!!!!licit ( 2 2 : ) 1 phlvf f lo Loppspofifef
=== GENERATED SEQUENCE 3 ===
 al ilt c c b gggogugsggbongngnadagagginggigiagorgorgongegygicalgicddnghongogeolooloosooseoso
=== GENERATED SEQUENCE 4 ===
> >? - + + + C C C,s * ~ about bit bits parts of in code programming with with using application from from at a some end End ; | { } \ now there Ther
=== GENERATED SEQUENCE 5 ===
@ @ : ( ) ) ) ) > < < < < < < < p awmmms -g gGddnLDdldsdosabbbed

In [ ]:
!python run_metrics.py \
  --model_type bert \
  --model_name_or_path "models/karpathy" \
  --test_data_file "data/karpathy/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 21:33:27 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='bert', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/karpathy/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Downloading: 100% 482/482 [00:00<00:00, 535kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 5.28MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 3.21MB

# Obama


In [ ]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/Obama" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --repetition_penalty 2.0\
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 21:14:30 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/Obama', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=2.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
 how very little small by orlma not toto do doing did could should need have had been was done accomplished with with of it that that much much many during ’'s s work working together Together now Now Mor
=== GENERATED SEQUENCE 2 ===
 Obama )s — – ” “ ‘'’ ”tt an ”'" … … ” ” ” ” "so Imagine me Me as As So I I am m headingraderade @ called Joe Joe442
=== GENERATED SEQUENCE 3 ===
the of question or Or answer Answer answered answering without with much little Little surprise surprises, — ‘ ” " “ – ’'' it is was always never ever Ever told asked America Americans in 

In [ ]:
!python run_metrics.py \
  --model_type bert \
  --model_name_or_path "models/Obama" \
  --test_data_file "data/BarackObama/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/01/2021 21:40:22 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/Obama', model_type='bert', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/BarackObama/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight'

# WSJ


In [ ]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/wsj" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --repetition_penalty 2.0\
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/02/2021 00:56:31 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=2.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
 - - - - - - - - w WWWSSIS GODEDECE NESETELE DEAEAHAHHHHOMOWymYy YXLLIFIV
=== GENERATED SEQUENCE 2 ===
AT ATMMs 3 Three ofof the The 5 five best greatestest in In Music Fi Fifi fimic to, come coming years year on on on : : : i @m mms & 
=== GENERATED SEQUENCE 3 ===
 & and 2 Two : : ; 1 4 6 0 33 - - +, 7 111220141316 2012 2012 2013 # # The Best best ofof 2016 2015 in at the the lates
=== GENERATED SEQUENCE 4 ===
TTTBBM AMINGING R RIMRR & and n thee @ _ _ _ call callsn # # w WSsJWowawayyyew
=== GENERATED SEQUENCE 5 ===
N R Y, — ‘ ’ ” “ ‘ “ – ” ” ” ” ’ ” ” ” ” ” ” ” 

In [ ]:
!python run_metrics.py \
  --model_type bert \
  --model_name_or_path "models/wsj" \
  --test_data_file "data/wsj/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/02/2021 00:56:53 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='bert', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/wsj/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.wei

# CNN


In [ ]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/cnn" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --repetition_penalty 2.0\
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/08/2021 06:49:15 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=2.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
 Why Do Did It it? : : & @ at At'risk s S Secretpoon Follow # # #a RaAhasashacha Cha Atadadasms 44XxIV V II
=== GENERATED SEQUENCE 2 ===
G B F L S R MMCH H DDSVS N NE U O IN ON,,, on off off to air Air withingings T t lweetweet 
=== GENERATED SEQUENCE 3 ===
 The No Next next Coming Girl Girls girls boys, — " " " Out out Find find the the weekend week'’ s s s theme Theme of to Thanksgiving Christmas Santa in In That this This Is is
=== GENERATED SEQUENCE 4 ===
 insideall : : 1 Oneppm pm PM ETETET TTSG G ZIFITCAAA AAA &,????.... 3! < " 2 Two - on off off
=== GENERAT

In [ ]:
!python run_metrics.py \
  --model_type bert \
  --model_name_or_path "models/cnn" \
  --test_data_file "data/cnn/test.txt" \
  --length 50 \
  --stop_token "<EOS>" \
  --k 50 \
  --num_ref 50

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
12/08/2021 06:49:37 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='bert', n_gpu=1, no_cuda=False, num_ref=50, output_dir='outputs', p=0.9, repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, test_data_file='data/cnn/test.txt')
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Downloading: 100% 482/482 [00:00<00:00, 465kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 4.31MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 2.50MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 6.01MB/s]
Downloading: 100% 1.33G/1.33G [00:43<00:00, 33.0MB/s]
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias